In [1]:
# 通过 bif 文件生成因果数据集 .npy
import numpy as np
import pandas as pd
from pgmpy.readwrite import BIFReader
from pgmpy.sampling import BayesianModelSampling
from pgmpy.models import BayesianNetwork

/home/wukaiwen/envs/miniconda3/envs/cdllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "../../models/asia.bif"
n_samples = 1000

In [4]:
# 加载网络模型
reader = BIFReader(model_path)
model = reader.get_model()

# 采样数据
inference = BayesianModelSampling(model)
data = inference.forward_sample(size=n_samples)

# 保存为 .npy 文件
# 可能需要把类别变量转换为数值型
data_numeric = data.apply(lambda x: pd.factorize(x)[0])
np.save(f"../asia_data_n{n_samples}.npy", data_numeric.values)

Generating for node: dysp: 100%|██████████| 8/8 [00:00<00:00, 1103.47it/s]


In [5]:
# 可选：保存为 CSV 文件以便查看
data_numeric.to_csv(f"../asia_data_n{n_samples}.csv", index=False)